In [1]:
import pandas as pd
import math
import numpy as np


In [2]:
df = catalog.load("intermediate_data")

2023-05-04 19:37:16,883 - kedro.io.data_catalog - INFO - Loading data from 'intermediate_data' (CSVDataSet)...


In [3]:
df.head()

,SRCID,IntDate,Mode,CF_Order,Q1,Q2,Q2_Oth,Q3,Q3_Oth,Q4,...,wt_design,wt_propensit,wt_base_anu,wt_wave_anu,weight_anu,p_gender_sdc,p_age_group_sdc,p_education_sdc,p_state_sdc,StateMap
0,39,2019-04-11,2,1,2.0,19.0,NaN,16.0,NaN,3.0,...,4695.4258,1.150989,5384.734620,853.713782,0.091853,2.0,3.0,1.0,1.0,1.0
1,156,2019-04-09,2,2,1.0,12.0,NaN,11.0,NaN,1.0,...,4997.3515,1.074450,5349.881695,908.609364,0.097759,2.0,4.0,1.0,1.0,1.0
2,605,2019-04-09,2,1,2.0,NaN,NaN,NaN,NaN,3.0,...,16529.8031,1.074450,17695.871707,32062.736887,3.449705,2.0,3.0,3.0,1.0,1.0
3,41,2019-04-18,2,1,2.0,16.0,NaN,5.0,NaN,3.0,...,6108.6230,1.150989,7005.395282,5064.031682,0.544851,2.0,4.0,2.0,1.0,1.0
4,1295,2019-04-18,1,1,4.0,29.0,NaN,16.0,NaN,5.0,...,4100.6080,1.150989,4702.594994,8231.010107,0.885594,1.0,2.0,2.0,1.0,1.0


In [4]:
columns = df.columns

In [5]:
columns

Index(['SRCID', 'IntDate', 'Mode', 'CF_Order', 'Q1', 'Q2', 'Q2_Oth', 'Q3',
       'Q3_Oth', 'Q4',
       ...
       'wt_design', 'wt_propensit', 'wt_base_anu', 'wt_wave_anu', 'weight_anu',
       'p_gender_sdc', 'p_age_group_sdc', 'p_education_sdc', 'p_state_sdc',
       'StateMap'],
      dtype='object', length=202)

In [6]:
columns.get_loc("S1")

136

In [7]:
s_col = columns[136:167]

In [8]:
s_col

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
       'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21',
       'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31'],
      dtype='object')

In [9]:
catalog.list()

['raw_data',
 'codebook',
 'intermediate_data',
 'processed_data',
 'r_labels',
 'processed_codebook',
 'feature_data',
 'parameters']

In [10]:
codebook_xls = pd.DataFrame(catalog.load("codebook"))

2023-05-04 19:37:17,069 - kedro.io.data_catalog - INFO - Loading data from 'codebook' (ExcelDataSet)...


In [11]:
labels = codebook_xls[['Variable', 'Position', 'Label']]

In [12]:
labels.dropna(inplace=True)

2023-05-04 19:37:17,605 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\3937756069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels.dropna(inplace=True)



## Converts the labels of questions S1-S31 into annual return percentages

In [13]:
s_labels = labels.iloc[136:167]

In [14]:
s_labels["fv"] = s_labels["Label"].str.split(' ').str[8].str[1:].astype(int)

2023-05-04 19:37:17,638 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\3137119574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_labels["fv"] = s_labels["Label"].str.split(' ').str[8].str[1:].astype(int)



In [15]:
s_labels["annual_return"] = (s_labels["fv"] - 300) / 300

2023-05-04 19:37:17,648 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\2339710980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_labels["annual_return"] = (s_labels["fv"] - 300) / 300



In [16]:
s_labels.head()

,Variable,Position,Label,fv,annual_return
724,S1,137.0,S1. Would you rather receive $300 today or $46...,462,0.54
728,S2,138.0,S2. Would you rather receive $300 today or $37...,375,0.25
732,S3,139.0,S3. Would you rather receive $300 today or $33...,336,0.12
736,S4,140.0,S4. Would you rather receive $300 today or $31...,318,0.06
740,S5,141.0,S5. Would you rather receive $300 today or $30...,309,0.03


## Converts the labels of questions R1-R31 into return values

In [17]:
r_labels = labels.iloc[104:135]

In [18]:
r_labels["ev_rf"] = r_labels["Label"].str.split(' ').str[33].str[1:].astype(int)
r_labels["ev_gamble"] = 450
r_labels["risk_premium"] = r_labels["ev_gamble"] - r_labels["ev_rf"]

2023-05-04 19:37:17,699 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\2002518525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_labels["ev_rf"] = r_labels["Label"].str.split(' ').str[33].str[1:].astype(int)

2023-05-04 19:37:17,702 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\2002518525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_labels["ev_gamble"] = 450

2023-05-04 19:37:17,704 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Lo

In [19]:
r_labels["eu_rf"] = r_labels["ev_rf"].apply(lambda x: x**0.5)
r_labels["eu_gamble"] = math.sqrt(900)*0.5
r_labels["eu_diff"] = r_labels["eu_rf"] - r_labels["eu_gamble"]
r_labels.head()

2023-05-04 19:37:17,713 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\1855085640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_labels["eu_rf"] = r_labels["ev_rf"].apply(lambda x: x**0.5)

2023-05-04 19:37:17,715 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\Temp\ipykernel_22628\1855085640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_labels["eu_gamble"] = math.sqrt(900)*0.5

2023-05-04 19:37:17,717 - py.warnings - WARNING - C:\Users\gdbt0\AppData\Local\

,Variable,Position,Label,ev_rf,ev_gamble,risk_premium,eu_rf,eu_gamble,eu_diff
587,R1,105.0,R1. What would you prefer: a 50 percent chance...,480,450,-30,21.908902,15.0,6.908902
591,R2,106.0,R2. What would you prefer: a 50 percent chance...,240,450,210,15.491933,15.0,0.491933
595,R3,107.0,R3. What would you prefer: a 50 percent chance...,120,450,330,10.954451,15.0,-4.045549
599,R4,108.0,R4. What would you prefer: a 50 percent chance...,180,450,270,13.416408,15.0,-1.583592
603,R5,109.0,R5. What would you prefer: a 50 percent chance...,210,450,240,14.491377,15.0,-0.508623


In [20]:
# Risk averse - utility alpha = 0.5
# Risk loving - utility alpha = 2
# Risk neutral - utility alpha = 1, coefficient 0.1

In [21]:
df[s_col].info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2054 entries, 0 to 2053
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   S1      2050 non-null   float64
 1   S2      1044 non-null   float64
 2   S3      744 non-null    float64
 3   S4      423 non-null    float64
 4   S5      206 non-null    float64
 5   S6      217 non-null    float64
 6   S7      318 non-null    float64
 7   S8      151 non-null    float64
 8   S9      167 non-null    float64
 9   S10     298 non-null    float64
 10  S11     238 non-null    float64
 11  S12     124 non-null    float64
 12  S13     114 non-null    float64
 13  S14     60 non-null     float64
 14  S15     36 non-null     float64
 15  S16     24 non-null     float64
 16  S17     1006 non-null   float64
 17  S18     670 non-null    float64
 18  S19     270 non-null    float64
 19  S20     66 non-null     float64
 20  S21     204 non-null    float64
 21  S22     399 non-null    float64
 22  